# Can we predict crime rates in a borough through demographic data?

The tools we will use are Python, R and h2o.

## Get the tools set up
Let's start off with Python and import the necessary modules.

In [ ]:
from __future__ import print_function
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets.widgets import interact, Text
from IPython.display import display
import feather

Now issue a *magic* instruction to have graphical output embedded in this notebook,
and set some defaults.

In [ ]:
# use the notebook definition for interactive embedded graphics
# %matplotlib notebook

# use the inline definition for static embedded graphics
%matplotlib inline 

rcParam = {
    'figure.figsize': (12,6),
    'font.weight': 'bold',
    'axes.labelsize': 20.0,
    'axes.titlesize': 20.0,
    'axes.titleweight': 'bold',
    'legend.fontsize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
}

for key in rcParam:
    mpl.rcParams[key] = rcParam[key]

This is a Python notebook, but we can embed other languages as well.

If we precede a code cell with `%%R` the rest will be handled by the R-interpreter.
For this to work, we need to load the `rpy2.ipython` extension.

In [ ]:
import rpy2
%load_ext rpy2.ipython

## Get the data
We will use 2 datasets:

* Crime figures

* Demographic data

In [ ]:
%%bash
wget -nv -nc https://github.com/nijmegenR/meetups/raw/master/2017-01-25-meetINN-meetup/data.zip
if [ -f data.zip ]; then unzip -n data.zip; fi

### Crime figures
The Dutch institute *CBS* (Centraal Bureau voor de Statistiek) provides crime figures for all 
municipalities, towns and even boroughs in the Netherlands.

It is provided as a Microsoft Excel spreadsheet with separate sheets for the years 2010 through to 2015.
You can download it from 
`https://www.cbs.nl/-/media/_excel/2016/45/Geregistreerde-criminaliteit-per-gemeente-wijk-en-buurt-2010-2015.xlsx`
or use the file provided.

In [ ]:
cbs_crime_url = (
#    "https://www.cbs.nl/-/media/_excel/2016/45/"
    "Geregistreerde-criminaliteit-per-gemeente-wijk-en-buurt-2010-2015.xlsx"
)

The names of the columns in the spreadsheet are provided in several header rows.
That's why we need to provide the column names separately.

In [ ]:
columns = [
    "Regiocode",
    "Regionaam",
    "Inwoners",
    "Vermogen, vernieling en geweld",
    "Vermogensmisdrijven",
    "Diefstal",
    "Fietsendiefstal",
    "Diefstal overige vervoersmiddelen",
    "Diefstal uit of vanaf vervoermiddelen",
    "Zakkenrollerij, straatroof en beroving",
    "Woninginbraak (incl schuur, garage(box) en tuinhuis)",
    "Diefstal/inbraak uit niet-residentiele gebouwen",
    "Overige diefstal/inbraak",
    "Overige vermogensmisdrijven",
    "Vernielingen en misdrijven tegen openbare orde en gezag",
    "Vernielingen",
    "Vernieling aan auto",
    "Overige vernieling",
    "Misdrijven tegen openbare orde en gezag",
    "Gewelds- en seksuele misdrijven",
    "Mishandeling",
    "Bedreiging en stalking",
    "Overige gewelds- en seksuele misdrijven",
    "Totaal vermogen, vernieling en geweld (rel)",
    "Vermogensmisdrijven (rel)",
    "Diefstal/inbraak woning (rel)",
    "Vernieling en openbare orde (rel)",
    "Gewelds- en seksuele misdrijven (rel)"
    ]

Now, define a function `read_crime()` to read in one of the sheets from the spreadsheet.

In [ ]:
def read_crime(url, year):
    sheetnames = {
        2010:'Tabel 1',
        2011:'Tabel 2',
        2012:'Tabel 3',
        2013:'Tabel 4',
        2014:'Tabel 5',
        2015:'Tabel 6'
        }
    try:
        sheet = sheetnames[int(year)]
    except KeyError:
        raise Exception("No crime data for the year {} available".format(year))
        
    crime_df = pd.read_excel(url, sheetname=sheet,
        header=None, names=columns, skiprows=6, na_values=".")
    # crime_df.fillna(0, inplace=True)
    return crime_df

Use our function to read in Dutch crime rates from 2015...

In [ ]:
crime_2015 = read_crime(cbs_crime_url, 2015)   

... and have peek

In [ ]:
crime_2015.head()

and basic statistics:

In [ ]:
crime_2015.describe()

### Demografic Data

Also from the *CBS* is a file `shape 2014 versie 30.zip`
with demographic features, as well as geographic date in `shape` format.

The zip file contains files for municipalities (gemeenten), districts (wijken) and boroughs (buurten).
We will only use the latter.
(This was done by unzipping the file and put the contained files `buurt_2014.*` in a folder `buurten`.)

In [ ]:
cbs_demographic = 'buurten'

Now, we'll create a function `read_shape()` that reads this data.

In [ ]:
def read_shape(shape_file):
    def center(geom):
        try:
            centroid = geom.representative_point().coords[:][0]
        except Exception as e:
            x_min, y_min, x_max, y_max = geom.bounds
            centroid = (x_min + x_max)/2, (y_min + y_max)/2
        return centroid
    
    df = gpd.read_file(shape_file)
    df['center'] = df['geometry'].apply(center)
    df = df.where(df > -9999999., gpd.np.NaN)
    df['Regiocode'] = df['BU_CODE']
    df.set_index(['GM_CODE', 'WK_CODE', 'BU_CODE'], inplace=True)
    return df

Read in the demographic data for all boroughs in the Netherlands...

In [ ]:
nederland = read_shape(cbs_demographic)

... and have a preview:

In [ ]:
nederland.head()

## Combining data

Next, combine our two datasets.
Both have a column `Regiocode`, which is a unique code per borough.
This column is used to join the datasets.

In [ ]:
nl_crime_2015 = nederland.merge(crime_2015, on='Regiocode')

## Exploring data

Wouldn't it be great if we had the tool to quickly explore our data?

Well, we have. With *ipywidgets* we can make a UI-wrapper around a simple plot function.

If we want to change a single variable we should have a function with just that variable.
In this case we want a UI in which we can select the feature to show.

In the code below you this function is called `do_plot()`.
If this function must have a single argument ('`feature`'), then how will we provide the dataset to
plot, and possibly any additional arguments?

This is done by the function `mk_feature_plot` that creates the function `do_plot()` on the fly,
and passes its variables to this created function.
In other words, `mk_feature_plot()` is a factory that creates functions.
The design pattern is called *closure*.

In [ ]:
def mk_feature_plot(dframe, **kwargs):
    new_kwargs = dict(
        cmap='OrRd',
        linewidth=0.2,
        label=False
    )
    new_kwargs.update(kwargs)

    def do_plot(feature):
        ax = dframe.plot(column=feature, **new_kwargs)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(feature)

        if new_kwargs['label']:
            for idx, row in dframe.iterrows():
                plt.annotate(s=row[feature], xy=row['center'],
                             horizontalalignment='center', fontsize=6)
    return do_plot

Create a subset with only the boroughs in Nijmegen.

In [ ]:
nijmegen = nl_crime_2015[nl_crime_2015['GM_NAAM']=='Nijmegen']

The function interact from ipywidgets expects as arguments a function and its arguments.

In this case the function has only one argument: `feature`.

In [ ]:
interact(
    mk_feature_plot(nijmegen, alpha=0.7,
                    label=True, cmap='seismic'),
    feature=list(nijmegen.columns));

We can also use a function that takes two arguments: `city` and `feature`.
Let's make the function factory `mk_city_feature()` that creates such functions.

In [ ]:
def mk_city_feature_plot(dframe, **kwargs):
    new_kwargs = dict(
        cmap='OrRd',
        linewidth=0.2,
        label=False
    )
    new_kwargs.update(kwargs)

    def do_plot(city, feature):
        df = dframe[dframe['GM_NAAM']==city]
        ax = df.plot(column=feature, **new_kwargs)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title("%s in %s" % (feature, city))

        if new_kwargs['label']:
            for idx, row in df.iterrows():
                plt.annotate(s=row[feature], xy=row['center'],
                             horizontalalignment='center', fontsize=6)
    return do_plot

Now use `interact()` to create a user interface with this function.

In [ ]:
interact(
    mk_city_feature_plot(nl_crime_2015, label=True,
                         alpha=0.7, cmap='seismic'),
    city="Arnhem Overbetuwe Nijmegen Beuningen Groesbeek".split(),
    feature=list(nl_crime_2015.columns));

## Data massaging

Strip the geometric data from the data set and write out in the very efficient `feather` format,

In [ ]:
nl_crime_2015_no_geo = nl_crime_2015.copy()

del nl_crime_2015_no_geo['geometry']
del nl_crime_2015_no_geo['center']

feather.write_dataframe(nl_crime_2015_no_geo, 'nl_crime_2015.fthr')

## Machine learning in R

The outcome is a model that fits the demographic variables to the  crime rate.
A General Linear Model (*GLM*) is created (although, arguably, better model types are applicable).

Finally, the model is used to predict crime rates in Nijmegen's boroughs, based on the demographic figures.

### First import the necessary libraries and start up the h2o machine learning cluster.

In [ ]:
%%R
library(h2o)
library(feather)
library(MASS)
localH2O = h2o.init(nthreads = -1)

### Pick up the `feather` dataset in R

In [ ]:
%%R
#Load the data and prepare for modeling
nld = read_feather("nl_crime_2015.fthr")
nld[sapply(nld, is.character)] <- lapply(nld[sapply(nld, is.character)], as.factor)

### Split it into a test set (Nijmegen) and a training set (the rest of the Netherlands).

In [ ]:
%%R
#make a Nijmegen subset
nijmegen = nld[nld$GM_NAAM=="Nijmegen",]
#make a subset of Netherland w/o Nijmegen
rest_van_nld = nld[nld$GM_NAAM!="Nijmegen",]

### Upload the data frames to h2o server

In [ ]:
%%R
#convert into h2o table type
nijmegen = as.h2o(nijmegen)
rest_van_nld = as.h2o(rest_van_nld)

### Fit a statistical model

In [ ]:
%%R
thecolumns = colnames(rest_van_nld)
poisson.fit = h2o.glm(x = thecolumns[1:190], y = "Vermogen, vernieling en geweld", rest_van_nld, family = "poisson")

### Use the trained model to make a prediction for Nijmegen

In [ ]:
%%R
prediction = predict(poisson.fit, nijmegen)
nijmegentotaal = nijmegen
nijmegentotaal$prediction = prediction

head(nijmegentotaal)

### Inspect the model

In [ ]:
%%R
summary(poisson.fit)

### Visualise the results on a map

#### Add shape data to dataframe

In [ ]:
%%R
#read shape info from shapeInfo.csv
shapeinfo = read.csv("shapeInfo.csv")

In [ ]:
%%R
# Merge data frames
nijmegentotaal = merge(shapeinfo, nijmegentotaal, by.x="BU_CODE", by.y="BU_CODE")

#### Plot the prediction

In [ ]:
%%R
library(ggplot2)
library(ggmap)
## Plot data
mapCenter <- geocode("Nijmegen")
Nijmegen <- get_map(c(lon=mapCenter$lon, lat=mapCenter$lat),zoom = 12)#, maptype = "terrain", source="stamen")
NijmegenMap <- ggmap(Nijmegen)
NijmegenMap <- NijmegenMap +
  geom_polygon(aes(x=long, y=lat, group=group, fill=prediction),
               size=.2 ,color='black', data=as.data.frame(nijmegentotaal), alpha=0.8) +
  scale_fill_gradient(low = "green", high = "red")
NijmegenMap

#### Plot the actual data

In [ ]:
%%R
## Plot data
nijmegentotaal$werkelijk = nijmegentotaal$Vermogen..vernieling.en.geweld
NijmegenMapWerkelijk <- ggmap(Nijmegen)
NijmegenMapWerkelijk <- NijmegenMapWerkelijk +
  geom_polygon(aes(x=long, y=lat, group=group, fill=werkelijk),
               size=.2 ,color='black', data=as.data.frame(nijmegentotaal), alpha=0.8) +
  scale_fill_gradient(low = "green", high = "red")
NijmegenMapWerkelijk

#### Plot the difference

In [ ]:
%%R
nijmegentotaal$verschil = nijmegentotaal$prediction - nijmegentotaal$werkelijk
NijmegenMapVerschil <- ggmap(Nijmegen)
NijmegenMapVerschil <- NijmegenMapVerschil +
  geom_polygon(aes(x=long, y=lat, group=group, fill=verschil),
               size=.2 ,color='black', data=as.data.frame(nijmegentotaal), alpha=0.8) +
  scale_fill_gradient(low = "green", high = "red")
NijmegenMapVerschil